In [ ]:
# %pip install python-dotenv


In [ ]:
# %pip install ipywidgets --upgrade


# RAG 기본 구현
이 노트북은 Elasticsearch와 Sentence Transformers를 사용하여 RAG 기반 시스템을 구현합니다.

In [1]:
import os  # 운영 체제와 상호작용하기 위한 모듈 (환경 변수 등)
import json  # JSON 형식 데이터를 처리하기 위한 모듈 (로드, 덤프 등)
from elasticsearch import Elasticsearch, helpers  # Elasticsearch 클라이언트 및 대량 작업을 위한 헬퍼 함수 가져오기
from sentence_transformers import SentenceTransformer  # 문장 임베딩을 생성하기 위한 SentenceTransformer 모델 가져오기
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 로드하기 위한 모듈


In [2]:
# # Elasticsearch 연결 확인
# print(es.ping())

In [3]:
# print(es.count(index="test")["count"])


## Sentence Transformer 모델 초기화

In [5]:
# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## 임베딩 및 Elasticsearch를 위한 함수 정의

In [6]:
# SentenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]  # 배치 크기만큼 문서 슬라이싱
        contents = [doc["content"] for doc in batch]  # 각 문서의 "content" 필드 추출
        embeddings = get_embedding(contents)  # "content" 필드로부터 임베딩 생성
        batch_embeddings.extend(embeddings)  # 생성한 임베딩을 리스트에 추가
        print(f'batch {i}')  # 현재 배치 번호 출력
    return batch_embeddings  # 모든 배치 임베딩 반환

In [7]:
# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)

# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)

# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)

## 검색 및 검색 함수

In [8]:
# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")

# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",  # 어떤 필드에서 검색할 것인가
        "query_vector": query_embedding.tolist(),  # 검색할 벡터
        "k": size,  # 최종 몇 개의 문서를 제출할 것인가
        "num_candidates": 100  # 각 샤드별로 몇 개의 후보군을 뽑을 것인가?
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

## Elasticsearch 및 인덱스 설정

In [9]:
es_username = "elastic"
es_password = "clLMwYXskYxQ*yLKQDHo"

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="./elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

# 색인을 위한 setting 설정
settings = {
    "analysis": {  # 텍스트 분석을 위한 설정
        "analyzer": {  # 분석기 정의
            "nori": {  # 'nori'라는 이름의 사용자 정의 분석기
                "type": "custom",  # 사용자 정의 분석기임을 나타냄
                "tokenizer": "nori_tokenizer",  # 토크나이저로 'nori_tokenizer' 사용 (한국어 전용 토크나이저)
                "decompound_mode": "mixed",  # 복합 명사를 혼합된 형태로 분리
                "filter": ["nori_posfilter"]  # 토큰 필터 적용 (품사 필터링 등)
            }
        },
        "filter": {  # 필터 정의
            "nori_posfilter": {  # 'nori_posfilter'라는 이름의 품사 필터 설정
                "type": "nori_part_of_speech",  # 한국어 품사 필터 타입 지정
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]  # 색인하지 않을 품사들을 지정
            }
        }
    }
}


mappings = {
    "properties": {  # 문서의 각 필드에 대한 특성을 정의하는 부분
        "content": {  # 문서 내용 필드 설정
            "type": "text",  # 데이터 타입을 'text'로 지정
            "analyzer": "nori"  # 한글 분석기 'nori' 사용 (토큰화를 통해 검색 효율성 증대)
        },
        "embeddings": {  # 벡터 임베딩 필드 설정
            "type": "dense_vector",  # 필드 타입을 'dense_vector'로 설정 (고차원의 벡터 값 저장용)
            "dims": 768,  # 벡터 차원을 768로 지정 (Sentence Transformer 모델에서 생성한 임베딩 크기)
            "index": True,  # 색인 가능 여부 설정 (이 벡터 필드로 검색 수행 가능)
            "similarity": "l2_norm"  # 유사도 계산 방식으로 'l2_norm' 사용 (벡터 간 거리 계산 방식)
        }
    }
}


{'name': 'instance-11767', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'eQCNannCTnKB9C6ibEvKTw', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [10]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

## 문서 색인 및 임베딩

In [11]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]  # 문서 파일 읽기
embeddings = get_embeddings_in_batches(docs)  # 문서 임베딩 생성

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()  # 문서에 임베딩 추가
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


## 쿼리 및 검색 예시

In [12]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [13]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)  # 역색인을 사용하여 'test_query'와 관련된 문서 3개를 검색


# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:  # 검색 결과에서 각 문서를 순회
    print('score:', rst['_score'], 'source:', rst['_source']["content"])  # 각 문서의 점수와 내용 출력


score: 33.566788 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 18.97676 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 18.325773 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.


In [14]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)  # 벡터 유사도를 사용하여 'test_query'와 유사한 문서 3개를 검색


# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:  # 벡터 유사도 검색 결과에서 각 문서를 순회
    print('score:', rst['_score'], 'source:', rst['_source']["content"])  # 각 문서의 점수와 내용 출력


score: 0.0047387844 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.004689847 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 0.004264912 source: 소행성대가 위치한 곳에는 행성이 없는 이유는 목성과의 공명으로 인해 물질이 모이는 것을 방해하여 행성을 형성하지 못했기 때문입니다. 소행성대는 태양계 내부 행성들과 외부 행성들 사이에 위치한 영역으로, 많은 소행성들이 모여있는 지역입니다. 하지만 목성과의 공명 현상으로 인해 소행성들이 목성의 중력에 영향을 받아 행성을 형성하지 못하고 분산되거나 파괴되는 경우가 많습니다. 따라서 소행성대는 행성이 형성되기 어려운 환경이라고 할 수 있습니다. 이러한 이유로 소행성대에는 행성이 없는 것입니다.


In [15]:
# !ps -ef | grep elasticsearch # daemon 2개인데 왜 실행되는 건지 모르겠음;;

daemon     16287       1  1 07:15 ?        00:02:30 /data/ephemeral/home/code/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-6680164188296002282 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## RAG 구현

준비된 검색엔진과 LLM을 활용하여 대화형 RAG 구현

In [ ]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI  # OpenAI API를 사용하기 위한 모듈 가져오기
import traceback  # 예외 발생 시 오류 추적을 위한 모듈 가져오기

# OPENAI_API_KEY.env 파일 생성 
# OPENAI_API_KEY = "your API key here"

# 특정 .env 파일 경로를 지정하여 환경 변수 로드
load_dotenv(dotenv_path='OPENAI_API_KEY.env')

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 이후 OpenAI API 키를 사용할 코드 작성
# print(openai_api_key)  # API 키를 확인하기 위한 출력 (실제 코드에서는 제거)

In [17]:
client = OpenAI()  # OpenAI 클라이언트 생성
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [18]:
# RAG 구현에 필요한 Question Answering을 위한 LLM 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",  # 함수 타입 설정
        "function": {
            "name": "search",  # 함수 이름 설정
            "description": "search relevant documents",  # 함수 설명 설정
            "parameters": {
                "properties": {  # 파라미터의 속성 설정
                    "standalone_query": {
                        "type": "string",  # 파라미터의 데이터 타입을 'string'으로 설정
                        "description": "Final query suitable for use in search from the user messages history."  # 사용자 메시지 히스토리로부터 검색에 사용할 최종 질의 설명
                    }
                },
                "required": ["standalone_query"],  # 필수 파라미터로 'standalone_query' 설정
                "type": "object"  # 파라미터 전체 타입을 'object'로 설정
            }
        }
    },
]


In [19]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}  # 초기 응답 딕셔너리 생성 (질의, 상위 결과, 참조, 답변 필드 포함)

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages  # 시스템 메시지와 사용자 메시지를 결합하여 msg 생성
    try:
        result = client.chat.completions.create(  # OpenAI 클라이언트의 chat completion 생성 호출
            model=llm_model,  # 사용할 LLM 모델 설정
            messages=msg,  # 대화 메시지 전달
            tools=tools,  # 사용할 함수 리스트 전달
            # tool_choice={"type": "function", "function": {"name": "search"}},  # 사용 가능한 함수 중 하나를 선택하는 옵션 (주석 처리됨)
            temperature=0,  # 응답의 무작위성 설정 (0이면 더 결정적인 응답 생성)
            seed=1,  # 무작위성 제어를 위한 시드 설정
            timeout=10  # 요청 타임아웃 시간 설정 (초 단위)
        )
    except Exception as e:
        traceback.print_exc()  # 예외 발생 시 오류 추적 출력
        return response  # 오류 발생 시 초기화된 빈 응답 반환


    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:  # LLM이 함수 호출을 요청한 경우
        tool_call = result.choices[0].message.tool_calls[0]  # 첫 번째 함수 호출 정보 가져오기
        function_args = json.loads(tool_call.function.arguments)  # 함수 인자를 JSON 형식으로 파싱
        standalone_query = function_args.get("standalone_query")  # standalone_query 추출

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = sparse_retrieve(standalone_query, 3)  # sparse_retrieve 함수를 호출하여 검색 결과 가져오기

        response["standalone_query"] = standalone_query  # standalone_query 응답에 추가
        retrieved_context = []
        for i, rst in enumerate(search_result['hits']['hits']):  # 검색된 문서들을 순회
            retrieved_context.append(rst["_source"]["content"])  # 검색된 문서의 content 필드 추가
            response["topk"].append(rst["_source"]["docid"])  # 검색된 문서의 docid 추가
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})  # 검색된 문서의 점수 및 내용 추가

        content = json.dumps(retrieved_context)  # 검색된 문서들의 content를 JSON 형식으로 변환
        messages.append({"role": "assistant", "content": content})  # assistant 역할로 검색된 컨텍스트 추가
        msg = [{"role": "system", "content": persona_qa}] + messages  # 질문 응답 프롬프트와 함께 메시지 결합
        try:
            qaresult = client.chat.completions.create(  # OpenAI 클라이언트를 사용해 최종 답변 생성
                model=llm_model,  # 사용할 언어 모델 설정 (여기서는 gpt-3.5-turbo-1106)
                messages=msg,  # 대화에 사용할 메시지 리스트 전달
                temperature=0,  # 생성된 응답의 무작위성 설정 (0은 더 결정적인 응답 생성)
                seed=1,  # 무작위성 제어를 위한 시드 설정
                timeout=30  # 요청에 대한 타임아웃 설정 (30초)
            )
        except Exception as e:
            traceback.print_exc()  # 예외 발생 시 오류 추적 출력
            return response  # 오류 발생 시 초기화된 응답 반환
        response["answer"] = qaresult.choices[0].message.content  # 최종 생성된 답변 응답에 추가


    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content  # 검색 없이 생성된 답변 추가

    return response  # 최종 답변 반환

## 평가

In [20]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):  # 평가 파일과 출력 파일을 입력으로 받음
    with open(eval_filename) as f, open(output_filename, "w") as of:  # 평가 파일 읽기 모드, 출력 파일 쓰기 모드로 열기
        idx = 0  # 평가 데이터의 인덱스를 0으로 초기화
        for line in f:  # 평가 파일의 각 줄을 순회
            # if idx > 5:
            #     break  # 특정 개수 이상 평가를 제한할 때 사용 (현재 주석 처리됨)
            j = json.loads(line)  # 평가 데이터의 한 줄을 JSON 형식으로 로드
            print(f'Test {idx}\nQuestion: {j["msg"]}')  # 현재 인덱스와 질문 출력
            response = answer_question(j["msg"])  # 질문에 대한 답변 생성
            print(f'Answer: {response["answer"]}\n')  # 생성된 답변 출력

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}  # 평가 결과 저장할 딕셔너리 생성
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')  # 평가 결과를 JSON 형식으로 출력 파일에 저장
            idx += 1  # 인덱스를 1 증가

In [21]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../data/eval.jsonl", "sample_submission.csv")  # 평가 파일 경로와 출력 파일명을 입력으로 하여 eval_rag 함수 호출

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류를 조사하기 위한 방법은 다양합니다. 일반적으로 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 분류합니다. 또한 나무의 특징을 토대로 분류하는 방법도 있습니다. 분류는 다양한 방법으로 이루어지며, 각각의 방법은 특정한 목적에 따라 활용됩니다. 따라서 나무의 분류를 위해서는 다양한 정보와 방법을 활용하여야 합니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 죄송합니다, 제가 현재 각 나라의 공교육 지출 현황에 대한 정보를 찾을 수 없습니다. 더 많은 정보를 제공해 주시면 도와드릴 수 있을 것 같습니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 다양한 원인으로 발생할 수 있습니다. 일반적으로 기억 상실증은 인지 기능의 손상으로 인해 발생하며, 인지 기능은 정보를 받아들이고 저장하고 검색하는 능력을 말합니다. 기억 상실증의 원인으로는 뇌의 손상, 인지 기능 저하, 신체적 질환, 정서적 스트레스, 약물 남용 등이 있습니다. 그러나 각 개인의 상황에 따라 원인이 다를 수 있으며, 정확한 원인을 파악하기 위해서는 전문가의 진단이 필요합니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학 버스는 학교 출퇴근 시간에 많은 사람들을 운송하여 교통체증을 완화시키고, 학교로 향하는 학생들과 학교에서 돌아오는 사람들을